### 종합실습2

Requests 혹은 Selenium 모듈을 이용해 다이닝코드 페이지로 접속한 다음,<br>
원하는 지역의 맛집100건을 검색하여 주피터 노트북 화면에 출력하거나 엑셀로 저장해 보세요.<br>
*검색어 예시 : '포항'(링크 : https://www.diningcode.com/)

#### requests모듈

In [ ]:
# 모듈 불러오기
import requests
from bs4 import BeautifulSoup
import pandas as pd

search_key = "포항"

# url 주소는 "더보기" 클릭 후, Fetch/XHR에서 list.php로 나타나는 url로 설정할 것
url = "https://im.diningcode.com/API/isearch/"
h= {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"}

results = []
for i in range(1,11):
    print("{}페이지 크롤링 중...".format(i))
    form_data = {
        "dc_flag": "1",
        "lat": "36.0175999",
        "lng": "129.3292453",
        "mode": "poi",
        "page": str(i),
        "query": search_key,
        "rn_search_flag": "on",
        "search_type": "poi_search",
        "size": "20"
        }

    # header정보 없으면 데이터 못 읽어옴
    res = requests.post(url, data=form_data, headers=h, verify=False)

    # json형태로 변환하기
    json_data = res.json()

    # 필요 데이터 찾기
    rests = json_data["result_data"]["poi_section"]["list"]

    for rest in rests:
        name = rest["nm"]
        addr = rest["addr"]
        phone = rest["phone"]
        menu = rest["category"]
        info = {
            "식당이름" : name,
            "식당주소" : addr,
            "식당연락처" : phone,
            "메뉴" : menu,
            }
        results.append(info)

# df로 전환하기
df = pd.DataFrame(results)

# 엑셀 파일로 저장하기
df.to_excel("./outputs/{}맛집정보.xlsx".format(search_key), index=True)

#### selenium모듈

In [ ]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome("chromedriver")

# 다이닝코드 사이트 접속
driver.get("https://www.diningcode.com/")
driver.implicitly_wait(3)

# 검색어 입력
driver.find_element(By.CSS_SELECTOR, "#txt_keyword").send_keys("포항")
driver.implicitly_wait(3)

# Enter 키 Press
driver.find_element(By.CSS_SELECTOR, "#txt_keyword").send_keys(Keys.ENTER)
driver.implicitly_wait(3)

# 더보기 클릭
driver.find_element(By.CSS_SELECTOR, "#btn_normal_list > a").click()
driver.implicitly_wait(3)
# driver.execute_script("location.href='/list.php?query={}';".format(search_key)

# 더보기 클릭
for i in range(1,11):
    more_btn = driver.find_element(By.CSS_SELECTOR, "#div_list_more")
    try:
        print("{}페이지 로드중입니다.".format(i))
        more_btn.click()
        # driver.execute_script("getMoreList();")
        time.sleep(3)
    except:
        print("모든 데이터를 로드하였습니다.")

# 로드된 페이지 beautifulsoup
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
datas = soup.select("#div_list > li > a.blink")

# 정보 정리해서 리스트형태로 저장
results = []
for data in datas:
    result_dict = {
        "식당이름" : data.select(".btxt")[0].text.split(". ")[1],
        "메뉴" : data.select(".stxt")[0].text,
        "특징" : data.select(".ctxt")[0].text
    }
    results.append(result_dict)
    add = data.select("span:nth-child(5)")[0].text
    if "경상북도" in add or "경북" in add:
        add2 = add.replace("경북", "경상북도")
        idx = add.find("경상북도")
        result_dict["주소"] = add[idx:]
    else:
        result_dict["주소"] = add

df = pd.DataFrame(results)
df_ex = df.to_excel("./outputs/포항 맛집 100건 검색_selenium.xlsx", index=True)
display(df)